In [1]:
from google.colab import files
data_to_load = files.upload()

Saving local.zip to local.zip


In [2]:
import zipfile
import os

zip_path = "local.zip"
extract_path = "my_local_model"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dossier extrait dans :", extract_path)


Dossier extrait dans : my_local_model


In [4]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# Chemin du modèle LoRA en local
lora_path = "my_local_model"

# Configuration pour charger le modèle en 8 bits
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Charger la configuration du modèle LoRA
config = PeftConfig.from_pretrained(lora_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map='auto',
    load_in_8bit=True,  # Chargement du modèle en 8 bits
    quantization_config=bnb_config  # Appliquer la configuration BitsAndBytesConfig
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Charger le modèle LoRA
qa_model = PeftModel.from_pretrained(model, lora_path)


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# # Chemin du modèle LoRA en local
lora_path = "my_local_model"

config = PeftConfig.from_pretrained(lora_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
qa_model = PeftModel.from_pretrained(model, lora_path)

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


In [31]:
import time
import json
import torch
import re
import ast

categories = [
    "Gaz à effet de serre",
    "Elevage et utilisation des terres",
    "Pêche et chasse intensives",
    "Pollution plastique",
    "Déforestation",
    "Surconsommation",
    "Catastrophes naturelles",
    "Réchauffement climatique/canicules",
    "Sécheresse",
    "Couche d'ozone",
    "Feu de forêt",
    "Tensions alimentaires/famines",
    "Perte eau douce",
    "Hausse des océans et fonte des glaces",
    "Conséquence sociale",
    "Acidification des océans",
    "Biodiversité",
    "Pollution",
    "Energie renouvelable et nucléaire",
    "Transports décarbonés",
    "Engagements politiques et entreprises",
    "Activisme écologique",
    "Solutions innovantes",
    "Comportement de consommation",
    "Reforestation",
]

def classification_up_to_k(text, tokenizer, qa_model, categories, k=3, retry_attempts=5, base_delay=10):
    """
    Classifie un texte dans une liste de k catégories avec un modèle Hugging Face (comme Falcon ou Llama).

    Arguments :
    - text : str, le texte à classifier.
    - tokenizer : tokenizer Hugging Face.
    - qa_model : modèle Hugging Face.
    - categories : list, liste des catégories possibles.
    - k : int, nombre maximum de catégories à retourner.
    - retry_attempts : int, nombre maximum de tentatives en cas d'erreur.
    - base_delay : int, temps d'attente initial (secondes) avant de retenter en cas d'erreur 429.

    Retourne :
    - dict : Un dictionnaire avec les k meilleures catégories et leurs scores.
    """
    # Préparer le prompt pour le modèle
    prompt_message = f"""
    Donne seulement les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable.

    ## Règles :
    - Réponds uniquement avec un JSON strict du format suivant : {{ "Classe_1": "categorie_1", "Classe_2": "categorie_2", "Classe_3": "categorie_3" }}
    - Les catégories doivent être dans {categories}.
    - Les classes sont ordonnées de la plus probable à la moins probable.

    Texte : "{text}"

    Réponse :
    """

    # Tokeniser l'entrée
    batch = tokenizer(prompt_message, return_tensors='pt')

    # Envoyer le batch sur le bon device
    device = qa_model.device
    batch = {k: v.to(device) for k, v in batch.items()}

    for attempt in range(retry_attempts):
            # Effectuer l'inférence avec le modèle
            with torch.cuda.amp.autocast():
                output_tokens = qa_model.generate(**batch, max_new_tokens=200)

            # Décoder la réponse
            response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()
            # Extraire uniquement la partie après "Réponse :"
            response = response.split("Réponse :\n")[-1].strip()

            # Recherche du dictionnaire entre accolades
            match = re.search(r'\{.*\}', response)

            if match:
                # Conversion de la chaîne en dictionnaire
                dict_str = match.group(0)
                result_dict = ast.literal_eval(dict_str)
                return result_dict

            else :
              return("Désolé pas de dictionnaire trouvé")

            #return response



# Exemple d'utilisation
text = """La fonte des glaciers s'accélère dans les Alpes, mettant en danger la faune locale et augmentant les risques de pénurie d'eau pour les populations en aval."""

result = classification_up_to_k(text, tokenizer, qa_model, categories, k=3)
print("Résultat de classification :", result)


<ipython-input-31-1b8b6340c90a>:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Résultat de classification : {'Classe_1': 'Gaz à effet de serre', 'Classe_2': 'Feu de forêt', 'Classe_3': 'Perte eau douce'}


#Inférence

In [32]:
from google.colab import files
data_to_load = files.upload()

Saving annotation_sous_thematiques.csv to annotation_sous_thematiques.csv


In [34]:
import pandas as pd
path = "./annotation_sous_thematiques.csv"
df = pd.read_csv(path)
df = df.tail(29)
df = df[df['label']==1]
df.shape

(29, 18)

In [35]:
df["prediction_label"] = None
for idx, row in df.iterrows():
    df.at[idx, "prediction_label"] = classification_up_to_k(row["description"], tokenizer, qa_model, categories, k=3)
    time.sleep(4)  # Attendre 4 secondes entre chaque requête

# df[f"prediction_label"] = df["description"].apply(lambda x: classification_up_to_k(x, client, categories))
df[df['prediction_label'].notna()].shape

<ipython-input-31-1b8b6340c90a>:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Sett

(29, 19)

In [36]:
df.shape

(29, 19)

In [39]:
df[['prediction_label']]

,prediction_label
112,"{'Classe_1': 'Gaz à effet de serre', 'Classe_2..."
113,"{'Classe_1': 'Couche d'ozone', 'Classe_2': 'Sé..."
114,"{'Classe_1': 'Gaz à effet de serre', 'Classe_2..."
115,{'Classe_1': 'Elevage et utilisation des terre...
116,"{'Classe_1': 'Pêche et chasse intensives', 'Cl..."
117,"{'Classe_1': 'Couche d'ozone', 'Classe_2': 'El..."
118,Désolé pas de dictionnaire trouvé
119,"{'Classe_1': 'Pêche et chasse intensives', 'Cl..."
120,"{'Classe_1': 'Pêche et chasse intensives', 'Cl..."
121,Désolé pas de dictionnaire trouvé


In [44]:
def safe_extract(x, key):
    """
    Fonction qui extrait une valeur de 'x' en s'assurant que :
    - 'x' est bien un dictionnaire
    - 'key' existe dans 'x'
    - La valeur associée à 'key' est une liste avec au moins 'index+1' éléments
    """
    try:
        # Si x est un dictionnaire et contient la clé, extraire l'index demandé
        if isinstance(x, dict):
            return x.get(key, [None, None])  # Valeur par défaut = None (pour NaN)
    except (TypeError, IndexError):
        return None  # Retourne None pour faciliter la suppression des NaN

# Extraction des labels et probabilités
for i in range(1, 4):
    df[f"prediction_label_{i}"] = df["prediction_label"].apply(lambda x: safe_extract(x, f"Classe_{i}"))

# Suppression des lignes avec NaN dans les colonnes créées
df = df.dropna(subset=[f"prediction_label_{i}" for i in range(1, 4)])


In [47]:
df[["description", "label_1", "label_2", "label_3", "prediction_label_1", "prediction_label_2", "prediction_label_3"]]

,description,label_1,label_2,label_3,prediction_label_1,prediction_label_2,prediction_label_3
112,Le débat pour les 110 km/h sur les autoroutes ...,gaz_effet_de_serre,engagement_politique_et_entreprises,comportement_consommateur,Gaz à effet de serre,Pêche et chasse intensives,Sécheresse
113,"Après un été de la Saint-Martin, que l’on appe...",perte_biodiversite,rechauffement_climatique_canicule,NaN,Couche d'ozone,Sécheresse,Perte eau douce
114,"Le climat étant bouleversé, les cultures souff...",perte_biodiversite,rechauffement_climatique_canicule,NaN,Gaz à effet de serre,Elevage et utilisation des terres,Pêche et chasse intensives
115,"Dans toute la France, de nombreux projets susc...",agriculture_et_utilisation_du_sol,activisme_eco,agriculture_et_utilisation_du_sol,Elevage et utilisation des terres,Pêche et chasse intensives,Hausse des océans et fonte des glaces
116,"À quelques jours du lancement de la COP 27, l'...",hausse_niveau_mer_fonte_glace,engagement_politique_et_entreprises,gaz_effet_de_serre,Pêche et chasse intensives,Perte eau douce,Sécheresse
117,"Tous les soirs cette semaine, nous vous présen...",agriculture_et_utilisation_du_sol,solution_innovante,comportement_consommateur,Couche d'ozone,Elevage et utilisation des terres,Pêche et chasse intensives
119,Découvrons un système malin et écolo de recycl...,solution_innovante,eau_potable,intrants_chimique_pollution_plastique,Pêche et chasse intensives,Pollution plastique,Hausse des océans et fonte des glaces
120,"À Charm el-Cheikh en Égypte, 200 pays sont att...",gaz_effet_de_serre,engagement_politique_et_entreprises,gaz_effet_de_serre,Pêche et chasse intensives,Pollution plastique,Sécheresse
122,"En Bretagne, le groupe Le Duff tente depuis 5 ...",activisme_eco,eau_potable,consequence_sociale,Pêche et chasse intensives,Sur 21 hectares de terres agricoles,Conséquence sociale
123,Une récente enquête a mis en lumière que notre...,consequence_sociale,NaN,NaN,Pêche et chasse intensives,Feu de forêt,Sécheresse
